In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from sklearn import svm, preprocessing
from sklearn.impute import SimpleImputer
import h2o
from h2o.automl import H2OAutoML

### Objective 

Because we consider the revenue a continuous variable, we call this a _regression_ problem. It is common to evaluate regression problems according to some deviation measure of the error (difference) between the predictions and the ground truth values. Typical choices are Mean Squared Error (MSE) and its square root, the Root Mean Squared Error (RMSE).

However, both of these measures are quite sensitive to extreme values and work best if the typical scale of prediction errors are consistent across the dataset. This is not likely to be the case here because the revenue variable varies a lot. This means that a, say 10%, prediction error would matter a lot more if it is for one of the higher-earning stores than for one of the lower-earning ones. Consequently, we will use a variation that takes a log transform of the target variable before computing prediction errors.

**TL;DR**: submissions for this problem will be evaluated according to the `Root Mean Squared Log Error` (RMSLE):

- $\text{RMSLE}(y, \hat{y}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (\log(1 + \hat{y}_i) - \log(1 + y_i))^2}$

In the equation above, $y_i$ corresponds to the ground truth value for datapoint $i$, $\hat{y}_i$ corresponds to the predicted value for datapoint $i$, and $n$ denotes the total number of datapoints (dimensionality of $y$, $\hat{y}$). See the cell below for an implementation.



In [23]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))

In [77]:
df1= pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/dataset_train_1.csv')
df2= pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/dataset_train_2.csv')


df_train = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/dataset_train_without_rev_features.csv')
df_test = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/dataset_test_without_rev_features.csv')





In [3]:
h2o.init(ip="127.0.0.1",port="54321")



Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O_cluster_uptime:,42 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_aminp_hqvdpk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.753 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [69]:
train_df = h2o.import_file("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/dataset_train_without_rev_features.csv")
test_df = h2o.import_file("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/dataset_test_without_rev_features.csv")





Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [72]:
train_df2 = train_df.drop(["store_id", "C1"])
train_df3 = train_df2.drop([0], axis = 0)
test_df2 = test_df.drop("C1")
test_df3 = test_df2.drop([0], axis = 0)
test_df3

store_id,grunnkrets_id_population_count,district_name_population_count,municipality_name_population_count,grunnkrets_name_num_kids,grunnkrets_name_num_kids+,grunnkrets_name_num_youths,grunnkrets_name_num_youthAdult,grunnkrets_name_num_adult,grunnkrets_name_num_adults+,grunnkrets_name_num_pensinors,grunnkrets_name_kids_%,grunnkrets_name_kids+_%,grunnkrets_name_youths_%,grunnkrets_name_youthAdult_%,grunnkrets_name_adult_%,grunnkrets_name_adults+_%,grunnkrets_name_pensinors_%,district_name_num_kids,district_name_num_kids+,district_name_num_youths,district_name_num_youthAdult,district_name_num_adult,district_name_num_adults+,district_name_num_pensinors,district_name_kids_%,district_name_kids+_%,district_name_youths_%,district_name_youthAdult_%,district_name_adult_%,district_name_adults+_%,district_name_pensinors_%,municipality_name_num_kids,municipality_name_num_kids+,municipality_name_num_youths,municipality_name_num_youthAdult,municipality_name_num_adult,municipality_name_num_adults+,municipality_name_num_pensinors,municipality_name_kids_%,municipality_name_kids+_%,municipality_name_youths_%,municipality_name_youthAdult_%,municipality_name_adult_%,municipality_name_adults+_%,municipality_name_pensinors_%,grunnkrets_name_couple_children_0_to_5_years,grunnkrets_name_couple_children_18_or_above,grunnkrets_name_couple_children_6_to_17_years,grunnkrets_name_couple_without_children,grunnkrets_name_single_parent_children_0_to_5_years,grunnkrets_name_single_parent_children_18_or_above,grunnkrets_name_single_parent_children_6_to_17_years,grunnkrets_name_singles,grunnkrets_name_%_dist_of_couple_children_0_to_5_years,grunnkrets_name_%_dist_of_couple_children_18_or_above,grunnkrets_name_%_dist_of_couple_children_6_to_17_years,grunnkrets_name_%_dist_of_couple_without_children,grunnkrets_name_%_dist_of_single_parent_children_0_to_5_years,grunnkrets_name_%_dist_of_single_parent_children_18_or_above,grunnkrets_name_%_dist_of_single_parent_children_6_to_17_years,grunnkrets_name_%_dist_of_singles,district_name_couple_children_0_to_5_years,district_name_couple_children_18_or_above,district_name_couple_children_6_to_17_years,district_name_couple_without_children,district_name_single_parent_children_0_to_5_years,district_name_single_parent_children_18_or_above,district_name_single_parent_children_6_to_17_years,district_name_singles,district_name_%_dist_of_couple_children_0_to_5_years,district_name_%_dist_of_couple_children_18_or_above,district_name_%_dist_of_couple_children_6_to_17_years,district_name_%_dist_of_couple_without_children,district_name_%_dist_of_single_parent_children_0_to_5_years,district_name_%_dist_of_single_parent_children_18_or_above,district_name_%_dist_of_single_parent_children_6_to_17_years,district_name_%_dist_of_singles,municipality_name_couple_children_0_to_5_years,municipality_name_couple_children_18_or_above,municipality_name_couple_children_6_to_17_years,municipality_name_couple_without_children,municipality_name_single_parent_children_0_to_5_years,municipality_name_single_parent_children_18_or_above,municipality_name_single_parent_children_6_to_17_years,municipality_name_singles,municipality_name_%_dist_of_couple_children_0_to_5_years,municipality_name_%_dist_of_couple_children_18_or_above,municipality_name_%_dist_of_couple_children_6_to_17_years,municipality_name_%_dist_of_couple_without_children,municipality_name_%_dist_of_single_parent_children_0_to_5_years,municipality_name_%_dist_of_single_parent_children_18_or_above,municipality_name_%_dist_of_single_parent_children_6_to_17_years,municipality_name_%_dist_of_singles,grunnkrets_id_lv1_desc_pop_per_num_stores,grunnkrets_id_lv2_desc_pop_per_num_stores,grunnkrets_id_lv3_desc_pop_per_num_stores,grunnkrets_id_lv4_desc_pop_per_num_stores,district_name_lv1_desc_pop_per_num_stores,district_name_lv2_desc_pop_per_num_stores,district_name_lv3_desc_pop_per_num_stores,district_name_lv4_desc_pop_per_num_stores,municipality_name_lv1_desc_pop_per_num_stores,municipality_name_lv2_desc

In [75]:
x = train_df3.columns 
y = "revenue"
x.remove(y)

aml = H2OAutoML(max_models=5, seed=1)
aml.train(x=x, y=y, training_frame= train_df3)

AutoML progress: |
18:40:05.712: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,6.39328,0.2645161,6.7912216,6.140243,6.4968615,6.358165,6.179908
mean_residual_deviance,159.75868,19.25999,179.18979,151.48695,177.15561,158.27531,132.68571
mse,159.75868,19.25999,179.18979,151.48695,177.15561,158.27531,132.68571
null_deviance,608161.3,76780.42,668847.06,560902.75,677428.5,635544.25,498083.9
r2,0.3234132,0.0272486,0.2971203,0.3012385,0.3143674,0.357229,0.3471108
residual_deviance,411872.34,59336.723,469835.66,391745.28,464324.88,408508.6,324947.3
rmse,12.620769,0.7704452,13.386179,12.308003,13.309981,12.580752,11.518929
rmsle,0.9209656,0.0084209,0.9345847,0.9149978,0.9235773,0.9170715,0.9145967


In [35]:

lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
best_model = aml.get_best_model()

lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance,algo
StackedEnsemble_AllModels_1_AutoML_3_20221101_174740,12.6435,159.858,6.40098,0.920381,159.858,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_3_20221101_174740,12.6474,159.956,6.44071,0.930628,159.956,StackedEnsemble
GBM_1_AutoML_3_20221101_174740,12.6581,160.227,6.46545,0.938588,160.227,GBM
GBM_2_AutoML_3_20221101_174740,12.7978,163.784,6.4559,0.926581,163.784,GBM
GBM_3_AutoML_3_20221101_174740,12.8612,165.41,6.48997,0.933386,165.41,GBM
DRF_1_AutoML_3_20221101_174740,13.2599,175.825,6.78495,0.95253,175.825,DRF
GLM_1_AutoML_3_20221101_174740,15.3728,236.322,8.33833,1.20441,236.322,GLM


In [36]:
m = h2o.get_model(lb[4,"model_id"])
m.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,is_chain,2.515641e+06,1.000000,0.272489
1,municipality_name_lv3_desc_pop_per_num_stores,3.968751e+05,0.157763,0.042989
2,municipality_name_lv1_desc_pop_per_num_stores,3.172685e+05,0.126118,0.034366
3,number_of_lv4_desc,2.941659e+05,0.116935,0.031863
4,municipality_name_lv4_desc_pop_per_num_stores,2.667421e+05,0.106033,0.028893
...,...,...,...,...
135,municipality_name_singles,5.822728e+02,0.000231,0.000063
136,municipality_name_single_parent_children_0_to_...,1.542499e+02,0.000061,0.000017
137,municipality_name_num_kids+,0.000000e+00,0.000000,0.000000
138,municipality_name_num_adult,0.000000e+00,0.000000,0.000000


In [74]:
prediction = aml.leader.predict(test_df3)
df = test_df3.cbind(prediction)
df = df.as_data_frame(use_pandas=True)
result = df.loc[:,("store_id", 'predict')]
submission = result.rename(columns = {"store_id": "id",  "predict" : "predicted"})
submission.to_csv("StackedEnsemble_AllModels_1_AutoML_3_20221101_174740.csv", index = False)

submission

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


,id,predicted
0,914206820-914239427-717245,5.152239
1,916789157-916823770-824309,10.660138
2,913341082-977479363-2948,3.633863
3,889682582-889697172-28720,12.778327
4,997991699-998006945-417222,7.958262
...,...,...
8572,917323003-917383529-844309,17.766906
8573,917353379-917411824-845904,3.734782
8574,917072302-917089248-833647,12.619422
8575,916960557-916993161-829908,6.314860
